In [3]:
import os, shutil
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import skimage
import pickle
import matplotlib.image as mpimg



from lxml import etree
from tqdm import tqdm

We'll first put all of the annotations (as PASCAL VOC) and images into `./data`

Then we'll create train/val sets in `./prepared_data`

# Dataset Prep

Currently, the annotations are present in a single monolithic file. We need to process it

In [4]:
cvat_xml = './data/CVAT.xml'
img_dir = './data/imgs/'
anno_dir = './data/annotations/'
prep_dir = './prepared_data/'



In [5]:
def parse_anno_file(cvat_xml):
    root = etree.parse(cvat_xml).getroot()
    anno = []
    for image_tag in root.iter('image'):
        image = {}
        for key, value in image_tag.items():
            image[key] = value
        image['shapes'] = []
        for poly_tag in image_tag.iter('polygon'):
            polygon = {'type': 'polygon'}
            for key, value in poly_tag.items():
                polygon[key] = value
                
            points = [tuple(map(float, p.split(','))) for p in polygon['points'].split(';')]
            points = np.array([(int(p[0]), int(p[1])) for p in points])
            polygon['points'] = points
            
            image['shapes'].append(polygon)
        for box_tag in image_tag.iter('box'):
            box = {'type': 'box'}
            for key, value in box_tag.items():
                box[key] = value
            box['points'] = "{0},{1};{2},{1};{2},{3};{0},{3}".format(
                box['xtl'], box['ytl'], box['xbr'], box['ybr'])
            
            points = [tuple(map(float, p.split(','))) for p in box['points'].split(';')]
            points = np.array([(int(p[0]), int(p[1])) for p in points])
            box['points'] = points
            
            image['shapes'].append(box)

        image['shapes'].sort(key=lambda x: int(x.get('z_order', 0)))
        
        image['height'] = np.int(image['height'])
        image['width'] = np.int(image['width'])
        anno.append(image)

    return anno

In [6]:
food_labels = ['HotMain',
               'GreenVeg', 
               'OrangeVeg',
               'Starch',
               'Bread',
               'Dessert',
               'Sandwich',
               'SaladBowl',
               'Juice',
               'FreshFruit',
               'MilkDrink',
               'Soup']
food_class_id = list(range(len(food_labels)))
food_map = {x:food_class_id[i] for i,x in enumerate(food_labels)}
               #'Other' Tray Plat

In [7]:
tray_labels = ['Tray', 'Plate']
tray_class_id = [1,2]
tray_map = {'Tray':1, 'Plate':2}

In [8]:
class_labels = ['HotMain',
               'GreenVeg', 
               'OrangeVeg',
               'Starch',
               'Bread',
               'Dessert',
               'Sandwich',
               'SaladBowl',
               'Juice',
               'FreshFruit',
               'MilkDrink',
               'Soup',
               'Tray',
               'Plate']
class_map = {x:i+1 for i,x in enumerate(class_labels)} 

In [9]:
def get_masks_food(anno, allowed_labels = food_labels):
    # Will only accept polygon types and labels in allowed labels
    polygon_points = []
    polygon_labels = []
    
    shapes = anno['shapes']
    for shape in shapes:
        if (shape['type']=='polygon') & (shape['label'] in allowed_labels):
            polygon_points.append(shape['points'].T)
            polygon_labels.append(shape['label'])
    
    n_instances = len(polygon_labels)
    
    mask = np.zeros([anno["height"], anno["width"], n_instances],
                    dtype=np.uint8)

    for i,p in enumerate(polygon_points):
        rr, cc = skimage.draw.polygon(p[1], p[0])
        mask[rr, cc, i] = 1
        
    # Handle occlusions. This just decides that the most recent shape is the topmost.
    occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
    for i in range(n_instances-2, -1, -1):
        mask[:, :, i] = mask[:, :, i] * occlusion
        occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        # For plates and trays, we should ensure that the plate always occludes the tray
        
    class_ids = np.array([food_map[label] for label in polygon_labels])
    return mask.astype(np.bool), class_ids.astype(np.int32)
        

In [10]:
def get_masks_tray(anno, allowed_labels = ['Tray', 'Plate']):
    # Will only Trays as boxes and Plates as polygons
    # Resolves zorder by assuming trays are on bottom
    
    shape_points = []
    shape_labels = []
    
    shapes = anno['shapes']
    for shape in shapes:
        if (shape['type']=='box') & (shape['label']=='Tray'):
            shape_points.append(shape['points'].T)
            shape_labels.append(shape['label'])
    for shape in shapes:
        if (shape['type']=='polygon') & (shape['label']=='Plate'):
            shape_points.append(shape['points'].T)
            shape_labels.append(shape['label'])
    
    n_instances = len(shape_labels)
    
    mask = np.zeros([anno["height"], anno["width"], n_instances],
                    dtype=np.uint8)

    for i,p in enumerate(shape_points):
        rr, cc = skimage.draw.polygon(p[1], p[0])
        mask[rr, cc, i] = 1
        
    # Handle occlusions. This just decides that the most recent shape is the topmost.
    occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
    for i in range(n_instances-2, -1, -1):
        mask[:, :, i] = mask[:, :, i] * occlusion
        occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        # For plates and trays, we should ensure that the plate always occludes the tray
        
    class_ids = np.array([tray_map[label] for label in shape_labels])
    return mask.astype(np.bool), class_ids.astype(np.int32)
        

In [11]:
def get_masks(anno):
    # Resolves zorder by assuming trays are on bottom
    # Plates are on top of trays
    # Food can be on top of trays or plates
    
    shape_points = []
    shape_labels = []
    
    shapes = anno['shapes']
    for shape in shapes:
        if (shape['type']=='box') & (shape['label']=='Tray'):
            shape_points.append(shape['points'].T)
            shape_labels.append(shape['label'])
    for shape in shapes:
        if (shape['type']=='polygon') & (shape['label']=='Plate'):
            shape_points.append(shape['points'].T)
            shape_labels.append(shape['label'])
    for shape in shapes:
        if (shape['type']=='polygon') & (shape['label'] in food_labels):
                shape_points.append(shape['points'].T)
                shape_labels.append(shape['label'])
    
    n_instances = len(shape_labels)
    
    mask = np.zeros([anno["height"], anno["width"], n_instances],
                    dtype=np.uint8)

    for i,p in enumerate(shape_points):
        rr, cc = skimage.draw.polygon(p[1], p[0])
        mask[rr, cc, i] = 1
        
    # Handle occlusions. This just decides that the most recent shape is the topmost.
    occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
    for i in range(n_instances-2, -1, -1):
        mask[:, :, i] = mask[:, :, i] * occlusion
        occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))
        
    class_ids = np.array([class_map[label] for label in shape_labels])
    return mask.astype(np.bool), class_ids.astype(np.int32)
        

# Create Dataset

In [12]:
def remove_dir_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

In [13]:
remove_dir_contents(prep_dir)
#os.makedirs(os.path.join(prep_dir, 'all'))
train_dir = os.path.join(prep_dir, 'train')
train_img_dir = os.path.join(prep_dir, 'train', 'imgs')
val_dir = os.path.join(prep_dir, 'val')
val_img_dir = os.path.join(prep_dir, 'val', 'imgs')
os.makedirs(train_img_dir)
os.makedirs(val_img_dir)

In [14]:
# Copy only images with annotations
anno_all = parse_anno_file(cvat_xml)
anno_imgs = [x['name'] for x in anno_all]

# Let's use the last 15% for validation
n_train = np.floor(len(anno_imgs)*0.85).astype(np.int)
n_val = len(anno_imgs)-n_train
anno_train = anno_all[:n_train]
anno_val = anno_all[n_train:]
anno_imgs_train = anno_imgs[:n_train]
anno_imgs_val = anno_imgs[n_train:]

# Copy images
for img in anno_imgs_train:
    shutil.copyfile(os.path.join(img_dir, img), os.path.join(train_img_dir, img))
for img in anno_imgs_val:
    shutil.copyfile(os.path.join(img_dir, img), os.path.join(val_img_dir, img))
    
# Copy annotations
with open(os.path.join(train_dir, 'annotations.pkl') , 'wb') as f:
    pickle.dump( anno_train, f )
with open(os.path.join(val_dir, 'annotations.pkl') , 'wb') as f:
    pickle.dump( anno_val, f )

# MRCNN

In [15]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('./', "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
from mrcnn.model import log

Using TensorFlow backend.


In [16]:
class FoodConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "healthhack"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 14  # background + 3 foods

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
config = FoodConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                27
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [17]:
class_labels = ['HotMain',
               'GreenVeg', 
               'OrangeVeg',
               'Starch',
               'Bread',
               'Dessert',
               'Sandwich',
               'SaladBowl',
               'Juice',
               'FreshFruit',
               'MilkDrink',
               'Soup',
               'Tray',
               'Plate']

In [18]:
class foodDataset(utils.Dataset):
    def load_food(self, dataset_dir, subset):
        self.add_class("healthhack",1,"HotMain")
        self.add_class("healthhack",2,"GreenVeg")
        self.add_class("healthhack",3,"OrangeVeg")
        self.add_class("healthhack",4,"Starch")
        self.add_class("healthhack",5,"Bread")
        self.add_class("healthhack",6,"Dessert")
        self.add_class("healthhack",7,"Sandwich")
        self.add_class("healthhack",8,"SaladBowl")
        self.add_class("healthhack",9,"Juice")
        self.add_class("healthhack",10,"FreshFruit")
        self.add_class("healthhack",11,"MilkDrink")
        self.add_class("healthhack",12,"Soup")
        self.add_class("healthhack",13,"Tray")
        self.add_class("healthhack",14,"Plate")
        
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        with open(os.path.join(dataset_dir, 'annotations.pkl'), 'rb') as f:
            annotations = pickle.load(f)
            
        for a in annotations:
            shapes = a["shapes"]
            image_path = os.path.join(dataset_dir,'imgs', a['name'])
            image = skimage.io.imread(image_path)
            height = a["height"]
            width = a["width"]
            
            self.add_image(
                "healthhack",
                image_id = a["id"],
                path = image_path,
                width = width,
                height = height,
                shapes = shapes
            )
        
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        if info["source"] != "healthhack":
            return super(self.__class__, self).load_mask(image_id)
        
        shape_points = []
        shape_labels = []
        
        shapes = info["shapes"]
        for shape in shapes:
            if (shape['type']=='box') & (shape['label']=='Tray'):
                shape_points.append(shape['points'].T)
                shape_labels.append(shape['label'])
        for shape in shapes:
            if (shape['type']=='polygon') & (shape['label']=='Plate'):
                shape_points.append(shape['points'].T)
                shape_labels.append(shape['label'])
        for shape in shapes:
            if (shape['type']=='polygon') & (shape['label'] in food_labels):
                    shape_points.append(shape['points'].T)
                    shape_labels.append(shape['label'])

        n_instances = len(shape_labels)

        mask = np.zeros([info["height"], info["width"], n_instances],
                        dtype=np.uint8)

        for i,p in enumerate(shape_points):
            rr, cc = skimage.draw.polygon(p[1], p[0])
            mask[rr, cc, i] = 1

        # Handle occlusions. This just decides that the most recent shape is the topmost.
        occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
        for i in range(n_instances-2, -1, -1):
            mask[:, :, i] = mask[:, :, i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))

        class_ids = np.array([class_map[label] for label in shape_labels])
        
        return mask.astype(np.bool), class_ids.astype(np.int32)
    
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "healthhack":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [19]:
def modelTrain(model, dataset_dir):
    """Train the model."""
    # Training dataset.
    dataset_train = foodDataset()
    dataset_train.load_food(dataset_dir, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = foodDataset()
    dataset_val.load_food(dataset_dir, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')


In [20]:
def modelTrainBody(model, dataset_dir):
    """Train the model."""
    # Training dataset.
    dataset_train = foodDataset()
    dataset_train.load_food(dataset_dir, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = foodDataset()
    dataset_val.load_food(dataset_dir, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network body")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/10,
                epochs=100,
                layers='all')


In [21]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
# Which weights to start with?
#init_with = "coco"  # imagenet, coco, or last
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                      exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

W0922 11:34:11.272201 139865405294336 deprecation_wrapper.py:119] From /home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0922 11:34:11.287616 139865405294336 deprecation_wrapper.py:119] From /home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0922 11:34:11.307283 139865405294336 deprecation_wrapper.py:119] From /home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0922 11:34:11.323610 139865405294336 deprecation_wrapper.py:119] From /home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is d

Re-starting from epoch 45


In [28]:
modelTrain(model, prep_dir)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/mahasen/healthhack2019/Mask_RCNN/logs/healthhack20190922T0713/mask_rcnn_healthhack_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrc

/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/engine/training_gen

Epoch 1/30
100/100 [==============================] - 120s 1s/step - loss: 1.7091 - rpn_class_loss: 0.0119 - rpn_bbox_loss: 0.3213 - mrcnn_class_loss: 0.4278 - mrcnn_bbox_loss: 0.3468 - mrcnn_mask_loss: 0.6013 - val_loss: 5.0073 - val_rpn_class_loss: 0.2027 - val_rpn_bbox_loss: 2.6592 - val_mrcnn_class_loss: 0.8020 - val_mrcnn_bbox_loss: 0.5278 - val_mrcnn_mask_loss: 0.8156
Epoch 2/30
100/100 [==============================] - 56s 565ms/step - loss: 1.2455 - rpn_class_loss: 0.0109 - rpn_bbox_loss: 0.3296 - mrcnn_class_loss: 0.2652 - mrcnn_bbox_loss: 0.2149 - mrcnn_mask_loss: 0.4249 - val_loss: 2.3313 - val_rpn_class_loss: 0.0326 - val_rpn_bbox_loss: 0.5703 - val_mrcnn_class_loss: 0.7459 - val_mrcnn_bbox_loss: 0.4018 - val_mrcnn_mask_loss: 0.5808
Epoch 3/30
100/100 [==============================] - 53s 535ms/step - loss: 0.8199 - rpn_class_loss: 0.0080 - rpn_bbox_loss: 0.1571 - mrcnn_class_loss: 0.1664 - mrcnn_bbox_loss: 0.1368 - mrcnn_mask_loss: 0.3516 - val_loss: 2.1514 - val_rpn_cla

In [30]:
model_path_head = os.path.join('./', "mask_rcnn_heha_head.h5")
model.keras_model.save_weights(model_path_head)

In [22]:
modelTrainBody(model, prep_dir)

Training network body

Starting at epoch 45. LR=0.0001

Checkpoint Path: /home/mahasen/healthhack2019/Mask_RCNN/logs/healthhack20190922T0713/mask_rcnn_healthhack_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (Batc

W0922 11:34:30.401466 139865405294336 deprecation_wrapper.py:119] From /home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/mahasen/anaconda3/envs/mrcnn/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse In

Epoch 46/100
100/100 [==============================] - 157s 2s/step - loss: 0.6386 - rpn_class_loss: 0.0035 - rpn_bbox_loss: 0.1405 - mrcnn_class_loss: 0.0920 - mrcnn_bbox_loss: 0.0909 - mrcnn_mask_loss: 0.3117 - val_loss: 1.2168 - val_rpn_class_loss: 0.0108 - val_rpn_bbox_loss: 0.2298 - val_mrcnn_class_loss: 0.4572 - val_mrcnn_bbox_loss: 0.2146 - val_mrcnn_mask_loss: 0.3045
Epoch 47/100
100/100 [==============================] - 76s 764ms/step - loss: 0.4759 - rpn_class_loss: 0.0047 - rpn_bbox_loss: 0.1159 - mrcnn_class_loss: 0.0789 - mrcnn_bbox_loss: 0.0767 - mrcnn_mask_loss: 0.1997 - val_loss: 1.6996 - val_rpn_class_loss: 0.0377 - val_rpn_bbox_loss: 0.4896 - val_mrcnn_class_loss: 0.4382 - val_mrcnn_bbox_loss: 0.2699 - val_mrcnn_mask_loss: 0.4642
Epoch 48/100
100/100 [==============================] - 76s 758ms/step - loss: 0.6200 - rpn_class_loss: 0.0034 - rpn_bbox_loss: 0.0967 - mrcnn_class_loss: 0.1279 - mrcnn_bbox_loss: 0.1123 - mrcnn_mask_loss: 0.2798 - val_loss: 1.3466 - val_r

Process ForkPoolWorker-29:
Process ForkPoolWorker-28:
Process ForkPoolWorker-19:
Process ForkPoolWorker-25:
Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-20:
Process ForkPoolWorker-1:
Process ForkPoolWorker-18:
Process ForkPoolWorker-35:
Process ForkPoolWorker-15:
Process ForkPoolWorker-38:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-11:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-40:
Process ForkPoolWorker-4:
Process ForkPoolWorker-17:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Process ForkPoolWorker-16:
T

KeyboardInterrupt: 